# Lab Setup

In [ ]:
import pandas as pd
import pygwalker as pyg
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE

In [ ]:
casualty = pd.read_csv('data/casualty_train.csv')


vehicle = pd.read_csv('data/vehicle_train.csv')


## Modify the settings

In [ ]:
pd.set_option('display.max_colwidth', 500)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# EDA Checklist 

## INITIAL EXPLORATION - CASUALTY

In [ ]:
casualty.head()

In [ ]:
casualty.tail()

## INITIAL EXPLORATION - VEHICLE

In [ ]:
vehicle.head()

In [ ]:
vehicle.tail()

## DESCRIBE & SHAPE

In [ ]:
casualty.shape

In [ ]:
casualty.describe()

In [ ]:
vehicle.shape

In [ ]:
vehicle.describe()

## MISSING VALUES?

In [ ]:
casualty.isnull().sum()

In [ ]:
vehicle.isnull().sum()

## DUPLICATES?

In [ ]:
vehicle.duplicated().sum()

In [ ]:
casualty.duplicated().sum()

## DTYPES

In [ ]:
vehicle.dtypes

In [ ]:
casualty.dtypes

## VISUALISATION

In [ ]:
# Create a histogram of the 'Age' column
gwalker_casualty = pyg.walk(casualty)


# MERGE THE VALUES

In [ ]:
merged_df = pd.merge(casualty, vehicle, on=['accident_reference', 'vehicle_reference'], how='inner')

merged_df.head()

In [ ]:
merged_df.shape

In [ ]:
merged_df.describe()

In [ ]:
df = merged_df

In [ ]:
df.head(150)

# SELECT ONLY THE ROWS THAT !-1

In [ ]:
# select only the rows that do not contain -1 values in any of the columns
df = df[(df != -1).all(axis=1)]

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
df.head(100)

# Features (x), Target (y)

In [ ]:
# map the casualty_severity column to binary values of 1 (for "fatal" and "serious") and 0 (for "slight")
#df.loc['target'] = df['casualty_severity'].map({1: 1, 2: 1, 3: 0})

# create a new dataframe with only the target values (i.e., the target column)
y = df[['casualty_severity']]

# create a new dataframe with only the feature values (i.e., all columns except casualty_severity and target)
X = df.drop(['casualty_severity', 'casualty_severity'], axis=1)

In [ ]:
y.head(150)

# Map target to binary 1 (fatal, serious) and 0 (slight).

In [ ]:
y = y.replace({1: 1, 2:1, 3:0})

In [ ]:
y.head(150)

In [ ]:
y.dtypes

In [ ]:
X.dtypes

In [ ]:
X.head(100)

In [ ]:
X = X.drop(['lsoa_of_casualty', 'lsoa_of_driver', 'generic_make_model', 'accident_reference'], axis=1)

In [ ]:
X.head()

In [ ]:
X.shape

# SVM 

# Load the test data & merge it

In [ ]:
casualty_test = pd.read_csv('data/casualty_test.csv')


vehicle_test = pd.read_csv('data/vehicle_test.csv')


In [ ]:
vehicle_test.tail()

In [ ]:
merged_test = pd.merge(casualty_test, vehicle_test, on=['accident_reference', 'vehicle_reference'], how='inner')

In [ ]:
merged_test.head()

In [ ]:
merged_test.shape

In [ ]:
df1 = merged_test

In [ ]:
df1.shape

# Drop the columns you do not need


In [ ]:
df1 = df1.drop(['lsoa_of_casualty', 'lsoa_of_driver', 'generic_make_model', 'accident_reference'], axis=1)

In [ ]:
df1.head()

In [ ]:
df1.shape

In [ ]:
new_data = df1

# TRAIN & PREDICT

In [ ]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Perform SMOTE oversampling on the training set
## This is done due to Class imbalance of the dataset
sm = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = sm.fit_resample(X_train, y_train)

# Creating the SVM model
svm = SVC(kernel='linear', C=1.0, gamma='auto')

# Training the model on the resampled training data
svm.fit(X_train_resampled, y_train_resampled)

# Predicting on the test set
y_pred = svm.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

In [ ]:
# Preprocess the new data...
new_data = scaler.transform(new_data)

# Predict the class labels of the new data
predicted_labels = svm.predict(new_data)

In [ ]:
# create a dataframe with predicted labels
df1 = pd.DataFrame(predicted_labels, columns=['predicted_labels'])

# save the df
df1.to_csv('out/predicts.csv', index=False)

In [ ]:
df1.tail()